In [ ]:
import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import os
import csv
import pandas as pd
import numpy
 
sys.path.append(".")

import config

In [ ]:
print(config)

In [ ]:
PROJECT_PATH = config.data_directory

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# try to not get banned
import random

USER_AGENTS = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Mozilla/5.0 (iPad; CPU OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H321 Safari/600.1.4',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:11.0) Gecko/20100101 Firefox/11.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36 OPR/58.0.3135.107']

def getLyric(title, user_agent=True):
    url = "https://www.azlyrics.com/lyrics/{}.html".format(title)
    if user_agent:
        return requests.get(url, headers={'User-Agent': random.choice(USER_AGENTS)})
    return requests.get(url)

In [ ]:
def create_file(title):
    file_path = '{}/{}.txt'.format(PROJECT_PATH, title)
    if not os.path.exists(file_path):
        result = getLyric(title)
        c = result.content
        soup = BeautifulSoup(c)
        ringtones = soup.find_all("div", class_="ringtone")
        
        if len(ringtones) > 0:
            ringtone = ringtones[0]
            html_lyrics = ringtone.find_next_sibling('div')
            cleanText = html_lyrics.get_text().replace('\r','').replace('\n',' ').replace("\'",'"')

            file_path = '{}/{}.txt'.format(PROJECT_PATH, title.replace('/','-'))

            fo = open(file_path, "w", encoding='utf-8')
            fo.write(cleanText)
            fo.close()
            return 1
        else:
            return 0
    else:
        print('File Already Exists')

In [ ]:
create_file('wizkhalifa/youngwildfree')

In [ ]:
getLyric('wizkhalifa/youngwildfree')

In [ ]:
def create_songs_file(name):
    csvData = [['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artist', 'artist_id', 'album', 'album_id', 'album_release_date', 'album_release_date_precision', 'album_total_tracks']]

    with open('{}/{}.csv'.format(PROJECT_PATH, name), 'w', newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(csvData)

    csvFile.close()

In [ ]:
create_songs_file('songs')

In [ ]:
def fetch_songs(offset, count, name = 'songs', limit = 50):
    file_path = '{}/{}.csv'.format(PROJECT_PATH, name)
    new_file = 'a'
    tracks = []
    if not os.path.exists(file_path):
        tracks = [['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artist', 'artist_id', 'album', 'album_id', 'album_release_date', 'album_release_date_precision', 'album_total_tracks']]
        new_file = 'w'
        
    with open(file_path, new_file, newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        for i in range(offset, count * limit, limit):
            track_results = sp.search(q='year:1990', type='track', limit=limit,offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                tracks.append([t['id'], t['name'], t['popularity'], t['duration_ms'], t['explicit'], t['artists'][0]['name'], t['artists'][0]['id'],  t['album']['name'], t['album']['id'], t['album']['release_date'], t['album']['release_date_precision'], t['album']['total_tracks']])
        writer.writerows(tracks)
    csvFile.close()


In [ ]:
fetch_songs(3, 1)

In [ ]:
df = pd.read_csv('{}/{}.csv'.format(PROJECT_PATH, 'songs'))

In [ ]:
df.head()

In [ ]:
pattern = r'[^a-z0-9A-z]'
df['name_az'] = df['name'].str.replace(pattern, '').str.lower()
df['artist_az'] = df['artist'].str.replace(pattern, '').str.lower()
df['has_lyrics'] = 0

In [ ]:
import time

for index, row in df.head(2).iterrows():
    print(row['name_az'], row['artist_az'])
    df.loc[index, 'has_lyrics'] = create_file('{}/{}'.format(row['artist_az'], row['name_az']))
    time.sleep(10)

In [ ]:
df.head()

In [1]:
import spacy
from spacy_cld import LanguageDetector
 
nlp = spacy.load('en')
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)
doc = nlp('This is some English text.')
 
doc._.languages  # ['en']
doc._.language_scores['en']  # 0.96

ModuleNotFoundError: No module named 'spacy_cld'